In [2]:
from tqdm import tqdm

from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.recharge.esp import ESPSettings
#from openff.recharge.esp.psi4 import Psi4ESPGenerator
#from optimize.openff_psi4_gen import Psi4ESPGenerator
from openff.recharge.esp.storage import MoleculeESPRecord, MoleculeESPStore
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.utilities.molecule import smiles_to_molecule
from openff.recharge.grids import GridSettingsType, GridGenerator

from openff.units.elements import SYMBOLS
from rdkit.Chem import AllChem
from openff.toolkit import Molecule
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

import os
import numpy as np
import random

import sys
sys.path.append('/mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4/')

from openff.toolkit import Molecule
from openff.recharge.esp import ESPSettings
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.grids import GridSettingsType, GridGenerator
from qcelemental.models.procedures import OptimizationInput, QCInputSpecification
from source.conformers.conformer_gen import Conformers
from openff.units import unit
from qcelemental.models.common_models import Model
from openff.recharge.utilities.molecule import smiles_to_molecule


In [2]:
#load our molecule to create the conformers

molecule = smiles_to_molecule("OCC(O)CO")
rdmol = molecule.to_rdkit()
AllChem.EmbedMultipleConfs(rdmol, numConfs=10, randomSeed=42)

In [3]:
#test the optimizer

optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
optimize_MMFF



0

In [4]:
#test all the conformers

for id in range(rdmol.GetNumConformers()):
    optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
    print(f'conformer:{id}, optimize (0 = converge, 1 = more iterations req): {optimize_MMFF}')

conformer:0, optimize (0 = converge, 1 = more iterations req): 0
conformer:1, optimize (0 = converge, 1 = more iterations req): 0
conformer:2, optimize (0 = converge, 1 = more iterations req): 0
conformer:3, optimize (0 = converge, 1 = more iterations req): 0
conformer:4, optimize (0 = converge, 1 = more iterations req): 0
conformer:5, optimize (0 = converge, 1 = more iterations req): 0
conformer:6, optimize (0 = converge, 1 = more iterations req): 0
conformer:7, optimize (0 = converge, 1 = more iterations req): 0
conformer:8, optimize (0 = converge, 1 = more iterations req): 0
conformer:9, optimize (0 = converge, 1 = more iterations req): 0


In [6]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol


In [7]:
#do the conformers look structurally sensible

for id in range(rdmol.GetNumConformers()):
    IPythonConsole.drawMol3D(rdmol,confId=id)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
optimize_MMFF

0

In [9]:
#now create a series of xyz files so we can run the conformers with ASE, first convert our conformers to xyz format and then write to file

from openff.units import unit

cwd = os.getcwd()

conformers = []



for confs in range(rdmol.GetNumConformers()):
    conformer = np.zeros((rdmol.GetConformer(confs).GetNumAtoms(), 3))
    for atom_index, coordinates in enumerate(rdmol.GetConformer(confs).GetPositions()):
        conformer[atom_index, :] = coordinates
    conformers.append(conformer * unit.angstrom)


for number ,conformer in enumerate(conformers):
    atoms = [
            {
                "element": SYMBOLS[atom.atomic_number],
                "x": conformer[index, 0],
                "y": conformer[index, 1],
                "z": conformer[index, 2],
            }
            for index, atom in enumerate(molecule.atoms)
        ]
    xyz = f'{molecule.n_atoms}\n{molecule.to_smiles()}\n'
    for row in atoms:
        xyz += f"{row['element']}\t{np.around(row['x'].magnitude,decimals=6)}\t{np.around(row['y'].magnitude,decimals=6)}\t{np.around(row['z'].magnitude, decimals=6)}\n"
    try:
        f = open(cwd+f"/conformer_{number}.xyz", 'x')
        f.write(xyz)
        f.close()
    except FileExistsError:
        continue

In [4]:


CWD = os.getcwd()

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Setup geometry optimisation
hf_model = Model(method="hf", basis="6-31G*")
spec = QCInputSpecification(model=hf_model, keywords={}, driver="gradient")
opt_spec = OptimizationInput(
            initial_molecule=qc_mol,
            input_specification=spec,
            keywords={"coordsys": "dlc", 
                      "program": "psi4"}                                        
        )

opt = qcengine.compute_procedure(opt_spec, "geometric")
print(f'geometry optimiztion was {opt.error}')
#return optimized molecule
optmized_mol = opt.final_molecule

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )

grid = GridGenerator.generate(test_mol, optmized_mol.geometry*unit.angstrom, grid_settings)

grid = grid.to(unit.angstrom).m
np.savetxt("grid.dat", grid, delimiter=" ", fmt="%16.10f")
#compute one-electron properties.
opt_input_2 = { "molecule" : optmized_mol,
              "driver" : "energy",
              "model" : {"method":"scf","basis":"6-31G*"},
              "protocols":{"wavefunction":"all","stdout":True,"native_files":"all"},
              "keywords":{"scf_properties":["GRID_ESP", "GRID_FIELD","MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"]}                               
              }


opt_2 = qcengine.compute(opt_input_2, "psi4", task_config={"scratch_directory":CWD,"scratch_messy":True})

print(opt_2.dict())

esp = (np.loadtxt("grid_esp.dat").reshape(-1, 1) * unit.hartree / unit.e)

electric_field = (np.loadtxt("grid_field.dat")* unit.hartree/ (unit.e * unit.bohr))

rdkit has generated 10 conformers from a requested 10 conformers
geometry optimiztion was None


/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


{'id': None, 'input_data': {'molecule': {'schema_name': 'qcschema_molecule', 'schema_version': 2, 'validated': True, 'symbols': array(['H', 'O', 'H'], dtype='<U1'), 'geometry': array([[-1.42427673, -0.3639913 ,  0.        ],
       [-0.00142791,  0.72236003,  0.        ],
       [ 1.42570464, -0.35836872,  0.        ]]), 'name': 'H2O', 'molecular_charge': 0.0, 'molecular_multiplicity': 1, 'connectivity': [(0, 1, 1.0), (1, 2, 1.0)], 'fix_com': True, 'fix_orientation': True, 'provenance': {'creator': 'QCElemental', 'version': '0.26.0', 'routine': 'qcelemental.molparse.from_schema'}, 'extras': {'canonical_isomeric_explicit_hydrogen_mapped_smiles': '[H:1][O:2][H:3]'}}, 'driver': 'energy', 'model': {'method': 'scf', 'basis': '6-31G*'}, 'protocols': {'wavefunction': 'all', 'stdout': True, 'native_files': 'all'}, 'keywords': {'scf_properties': ['GRID_ESP', 'GRID_FIELD', 'MULLIKEN_CHARGES', 'LOWDIN_CHARGES', 'DIPOLE', 'QUADRUPOLE', 'MBIS_CHARGES']}, 'provenance': {'cpu': 'Intel(R) Xeon(R) CPU 

In [20]:
psi4.core.variables(['MBIS Charges'])

{'CURRENT ENERGY': -76.00688623049868,
 'CURRENT REFERENCE ENERGY': -76.00688623049868,
 'DD SOLVATION ENERGY': 0.0,
 'HF KINETIC ENERGY': 75.71208232170089,
 'HF POTENTIAL ENERGY': -151.71896855219958,
 'HF TOTAL ENERGY': -76.0068862304987,
 'HF VIRIAL RATIO': 2.003893749844908,
 'NUCLEAR REPULSION ENERGY': 8.941189202753957,
 'ONE-ELECTRON ENERGY': -122.62733060212095,
 'PCM POLARIZATION ENERGY': 0.0,
 'PE ENERGY': 0.0,
 'SCF ITERATION ENERGY': -76.00688623049868,
 'SCF ITERATIONS': 8.0,
 'SCF TOTAL ENERGY': -76.00688623049868,
 'TWO-ELECTRON ENERGY': 37.679255168868295,
 'HF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ]),
 'HF QUADRUPOLE': array([[-2.78959587e+00, -8.82137120e-04,  0.00000000e+00],
        [-8.82137120e-04, -4.43606969e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -5.41213293e+00]]),
 'SCF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ])}

In [7]:
from source.optimize.openff_psi4_gen import Psi4Generate
import psi4

test_mol =  smiles_to_molecule('[H]O[H]')

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )


esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

xyz = Psi4Generate.run_calc(test_mol,conformer_list[0],esp_settings,compute_esp=True,compute_field=True, minimize=False)

molecule = psi4.geometry(
"""
3
[H]O[H]
H	-0.81024874	-0.18571238	-0.0
O	-0.00248133	0.36959931	-0.0
H	0.81273007	-0.18388693	0.0
"""
)

molecule.set_molecular_charge(0)
molecule.set_multiplicity(1)

E, wfn = psi4.energy('hf/6-31g*', return_wfn=True, molecule = molecule)
psi4.prop('hf/6-31G*', properties=["GRID_ESP", "GRID_FIELD","MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"])
#psi4.core.print_variables()
psi4.core.variables(['MBIS Charges'])
wfn.atomic_point_charges()

rdkit has generated 10 conformers from a requested 10 conformers
3
[H]O[H]
H	-0.81024874	-0.18571238	-0.0
O	-0.00248133	0.36959931	-0.0
H	0.81273007	-0.18388693	0.0


Scratch directory: /mnt/storage/nobackup/nca121/scratch_dir/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  5, 4, 3
    Auxiliary basis highest AM E, G, H:  6, 5, 4
    Onebody   basis highest AM E, G, H:  6, 5, 4
    Solid Harmonics ordering:            gaussian

*** tstart() called on login02.cluster
*** at Fri Oct 27 10:21:54 2023

   => Loading Basis Set <=

    Name: 6-31G*
    Role: ORBITAL
    Keyword: BASIS
    atoms 1, 3 entry H          line    44 file /mnt/nfs/home/nca121/mambaforge/envs/openff/share/psi4/basis/6-31gs.gbs 
    atoms 2    entry O          line   145 file /mnt/nfs/home/nca121/mambaforge/envs/openff/share/psi4/basis/6-31gs.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy S

/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


     A'        15      15 
     A"         4       4 
   -------------------------
    Total      19      19
   -------------------------

  ==> Iterations <==

                           Total Energy        Delta E     RMS |[F,P]|

   @DF-RHF iter SAD:   -75.47872758237143   -7.54787e+01   0.00000e+00 
   @DF-RHF iter   1:   -75.93623259853292   -4.57505e-01   2.61459e-02 ADIIS/DIIS
   @DF-RHF iter   2:   -75.98723851052344   -5.10059e-02   1.55582e-02 ADIIS/DIIS
   @DF-RHF iter   3:   -76.00630437270181   -1.90659e-02   1.94502e-03 ADIIS/DIIS
   @DF-RHF iter   4:   -76.00686035146124   -5.55979e-04   3.48966e-04 ADIIS/DIIS
   @DF-RHF iter   5:   -76.00688476572068   -2.44143e-05   6.83772e-05 DIIS
   @DF-RHF iter   6:   -76.00688618090938   -1.41519e-06   1.17327e-05 DIIS
   @DF-RHF iter   7:   -76.00688622888111   -4.79717e-08   2.17472e-06 DIIS
   @DF-RHF iter   8:   -76.00688623049868   -1.61756e-09   3.93880e-07 DIIS
  Energy and wave function converged.


  ==> Post-Iterations <

: 

In [ ]:
wfn
